Import du module `Machine`, qui permet d'instancier une machine supportée par Miasm (x86, arm, mips, ...)

In [1]:
from miasm2.analysis.machine import Machine
Machine.available_machine()

['arml',
 'armb',
 'armtl',
 'armtb',
 'sh4',
 'x86_16',
 'x86_32',
 'x86_64',
 'msp430',
 'mips32b',
 'mips32l',
 'aarch64l',
 'aarch64b']

In [ ]:
machine = Machine("x86_32")
print machine

`Machine` propose plusieurs interfaces:
* jitter
* dis_engine
* ir
* ira
* mn

In [ ]:
jitter = machine.jitter("gcc")
print jitter

Manipulation des registres

In [ ]:
jitter.cpu.EAX

In [ ]:
jitter.cpu.EAX = 1

In [ ]:
jitter.cpu.EAX

In [ ]:
regs = jitter.cpu.get_gpreg()

In [ ]:
print regs

Manipulation de la mémoire

In [ ]:
jitter.vm

In [ ]:
from miasm2.jitter.csts import PAGE_READ, PAGE_WRITE

In [ ]:
jitter.vm.add_memory_page(0x1000, PAGE_READ|PAGE_WRITE, "\x00"*0x1000, "test page")

In [ ]:
jitter.vm

In [ ]:
jitter.vm.get_mem(0x1000, 0x10)

In [ ]:
jitter.vm.set_mem(0x1000, "toto")

In [ ]:
jitter.vm.get_mem(0x1000, 0x10)

In [ ]:
jitter.get_str_ansi(0x1000)

In [ ]:
jitter.vm.set_mem(0x1000, "B844332211C3".decode('hex'))

In [ ]:
jitter.jit.log_mn = True
jitter.jit.log_regs = True

In [ ]:
jitter.init_run(0x1000)
jitter.continue_run()

In [ ]:
# Errors and stdout not displayed in jupyter => Go to native